# Scratchpad for  I<sub>t, θ</sub> and J<sub>t, θ</sub>

[Motivation](https://terrytao.wordpress.com/2018/03/18/polymath15-sixth-thread-the-test-problem-and-beyond/)
This leaves the final range $N \leq 10$ (roughly corresponding to $x \leq 1600$). Here we can numerically evaluate $H_t(x+iy)$ to high accuracy at a fine mesh (see the data here), but to fill in the mesh we need good upper bounds on $H'_t(x+iy)$. It seems that we can get reasonable estimates using some contour shifting from the original definition of $H_t$ (see here). We are close to finishing off this remaining region and thus solving the toy problem. 

## $I_{t,\theta}$

### Preliminary math

Reference 1: [thread 2](https://terrytao.wordpress.com/2018/02/02/polymath15-second-thread-generalising-the-riemann-siegel-approximate-functional-equation/).

$H_t(z) := \int_0^\infty e^{tu^2} \Phi(u) \cos(zu)\ du,$

and $\Phi(u) := \sum_{n=1}^\infty (2\pi^2 n^4 e^{9u} - 3\pi n^2 e^{5u} ) \exp(-\pi n^2 e^{4u}),$

Shifting the contour and exchanging order of summation and integration,

$H_t(z) = \frac{1}{2} (K_{t,\theta}(z) + \overline{K_{t,\theta}(\overline{z})}) \quad (A)$

where $K_{t,\theta}(z) := \sum_{n=1}^\infty (2\pi^2 n^4 I_{t,\theta}(z-9i, \pi n^2) - 3\pi n^2I_{t,\theta}(z-5i, \pi n^2))$

with $I_{t,\theta}(b,\beta) := \int_{i\theta}^{i\theta+i\infty} \exp(tu^2 - \beta e^{4u} + ibu)\ du \quad (B)$

where $\beta = \pi n^2$,

$-\pi/8 < \theta < \pi/8$,

and $b = z-9i$ or $b = z-5i$ or, in the case of a conjugated argument $\overline{z}$ as in (A) above, $b=\overline{z}-9i$ or $b = \overline{z}-5i$. In general, $b = x + (y-m)i$ where $z = x+iy$.

Letting $u = σ+iθ$, the exponent in (B) is

$t(σ+iθ)^2-βe^{4(σ+iθ)}+ib(σ+iθ) = t(σ+iθ)^2-βe^{4(σ)}(\cos(4\theta)+i\sin(4\theta)+i(x+i(y-m))(σ+iθ)$

```python
from sympy import *
init_printing(use_unicode=True)
t,σ,θ,β,b,e,x,y,m = symbols('t σ θ β b e x y m')
collect(expand(t*(σ+I*θ)**2-β*e**(4*(σ))*(cos(4*θ)+I*sin(4*θ))+I*(x+I*(y-m))*(σ+I*θ)),I)
```

$= - e^{4 σ} β \cos{\left (4 θ \right )} + m σ - t θ^{2} + t σ^{2} - x θ - y σ + i \left(- e^{4 σ} β \sin{\left (4 θ \right )} + m θ + 2 t θ σ + x σ - y θ\right)$



Let

$\operatorname{logmag}(σ) = - e^{4 σ} β \cos{\left (4 θ \right )} + m σ - t θ^{2} + t σ^{2} - x θ - y σ$

and $ω(σ) = \left(- e^{4 σ} β \sin{\left (4 θ \right )} + m θ + 2 t θ σ + x σ - y θ\right)$

Then the integrand is

$e^{\operatorname{logmag}(σ)}(\cos(ω(σ)) + i\sin(ω(σ)))$

### Preliminary code for the series in I<sub>t, θ</sub>

#### Bounding I<sub>t, θ</sub>

Reference 2: [Bounding the derivative 2](http://michaelnielsen.org/polymath1/index.php?title=Bounding_the_derivative_of_H_t_-_second_approach)

From reference 2, equation (0),

$$H_t(z) = \frac{1}{2} \sum_{n=1}^\infty \left[ 2\pi^2 n^4 I_{t,\theta}(x-(9-y)i, \pi n^2) - 3\pi n^2 I_{t,\theta}(x-(5-y)i, \pi n^2) + 2\pi^2 n^4 \overline{I_{t,\theta}(x-(9+y)i, \pi n^2)} - 3\pi n^2 \overline{I_{t,\theta}(x-(5+y)i, \pi n^2)} \right]$$

where $\beta = \pi n^2$ for $n = 1, \ldots, \infty$,

and $b = x-ia$ for $a = 9+y, 9-y, 5+y, 5-y$.

Note that if the argument to $K_{t,\theta}$ is $z$ or $\overline{z}$ as in (A) above, then it has the form $x \pm iy$. Hence if $b = x \pm iy-mi$ then $b = x - i(m \mp y) = x - ia$ where $a = m \mp y$ for $m = 9, 5$. This is consistent with usage in equation (0). Thus $m = 9, 5$ should be an explicit argument for code implementing $I_{t,\theta}$.

Reference 2 points out that when $x,y > 0$, and $b=x-(9+y)i$, the $ibu$ term of the exponent in the integrand of $I_{t,\theta}$ (equation B above) will be $(9+y)u + ixu$ which would be expected to dominate cases $a=9-y$, $a=5+y$, and $a=5-y$. Considering the dominant case, the reference favors 

$\theta := \frac{\pi}{8} - \frac{1}{4} \arctan \frac{y+9}{x} \quad (1)$

which is the imaginary part of the point $u_{inflection}$ at which $\frac{d}{du} \left(-\beta e^{4u} + ibu \right)= 0$ and which should supress much of the oscillation of the integrand in (B).

In [1]:
using DBNUpperBound
using DBNUpperBound.Asymptotics
using QuadGK

""" θ_default(y)

    Returns a value for θ which should cancel most oscillation in Itθ or Jtθ.
    """
θ_default(y) = π/8 - 1/4*atan((abs(y)+9)/abs(y)) # function definition

θ_default

Recall that the integrand is

$e^{\operatorname{logmag(σ)}}(\cos(ω(σ)) + i\sin(ω(σ)))$

where $\operatorname{logmag}(σ) = - e^{4 σ} β \cos{\left (4 θ \right )} + m σ - t θ^{2} + t σ^{2} - x θ - y σ$

and $ω(σ) = \left(- e^{4 σ} β \sin{\left (4 θ \right )} + m θ + 2 t θ σ + x σ - y θ\right)$

In [2]:
function logmag(x,y,t,σ,β,θ,m)
    return -e^(4*σ)*β*cos(4*θ)+m*σ-t*θ^2+t*σ^2-x*θ-y*σ
end

function ω(x,y,t,σ,β,θ,m)
    return -e^(4*σ)*β*sin(4θ)+m*θ+2*t*θ*σ+x*σ-y*θ
end

""" Itθ(x, y, n, m; t=.4, lower_limit=big(0.0), upper_limit=big(Inf), θ = θ_default(y))

    Estimates Itθ by adaptive Gauss-Kronrod quadrature, returning Itθ and an error estimate.
    NOTE: GK quadrature will fail for oscillatory arguments--larger values of x.

    The quadrature function uses multiprecision arithmetic if at least one limit of integration is multiprecision.

    If a very unhelpful diagnostic appears, increase precision. It is 256 bits by default. 
    """
function Itθ(x, y, n, m; t=.4, lower_limit=big(0.0), upper_limit=big(Inf), θ = θ_default(y))
        β = π*n^2
        mag(σ) = exp(logmag(x,y,t,σ,β,θ,m))
        ω1(σ) = ω(x,y,t,σ,β,θ,m)
        return quadgk((σ)->mag(σ)*(cos(ω1(σ))+im*sin(ω1(σ))), lower_limit, upper_limit) # return quadrature, error estimate
end

Itθ

In [3]:
# with precision at 90 bits
setprecision(90) do
    @show Itθ(1600.0, .4, 10, 9); println() # integrate to infinity 
    @show Itθ(1600.0, .4, 10, 9, upper_limit=big(1.0)); println() # integrate to 1
    @show Itθ(1600.0, .4, 10, 9, lower_limit=big(1.0)); println() #integrate from 1 to ∞
    @show Itθ(1e-3, .4, 10, 9); println() # integrate to infinity
    @show Itθ(1e-3, .4, 10, 9, upper_limit=1.0); println() # integrate to 1
    @show Itθ(1e-3, .4, 10, 9, lower_limit=big(1.0)); println() #integrate from 1 to ∞
end


Itθ(1600.0, 0.4, 10, 9) = (9.8140498435429387227726342045e-148 + 1.9123433943074696906381480567e-148im, 1.2201853092735775340508232483e-161)

Itθ(1600.0, 0.4, 10, 9, upper_limit=big(1.0)) = (9.8140498435429387227612003897e-148 + 1.9123433943074696906420022356e-148im, 1.1381664593968785600159537605e-161)

Itθ(1600.0, 0.4, 10, 9, lower_limit=big(1.0)) = (-1.2204921589139392950946092139e-7451 - 8.810493065410320739975412613e-7452im, 1.6316174983215494279584638591e-7465)

Itθ(0.001, 0.4, 10, 9) = (2.8569925607258333894159260564e-140 - 2.6153121031580823038065483558e-140im, 7.2361467178768999685423236508e-154)

Itθ(0.001, 0.4, 10, 9, upper_limit=1.0) = (2.8569925607258333894159252559e-140 - 2.6153121031580823038065499669e-140im, 7.8164184304246615031224133864e-154)

Itθ(0.001, 0.4, 10, 9, lower_limit=big(1.0)) = (3.4796126389279638137573026447e-7444 - 1.177864925509017539219660022e-7444im, 4.0243060738995228819185731034e-7458)



$I_{t,\theta,\geq X}(b,\beta) := \int_{i\theta+X}^{i\theta+\infty} \exp( tu^2 - \beta e^{4u} + ibu)\ du.$

When $\beta e^{4X} \cos(4\theta) \gt \max( \frac{t}{2}, \frac{a+2tX}{4}) \quad (2)$

$|I_{t,\theta,\geq X}(b,\beta)| \leq \frac{\exp( -t \theta^2 + tX^2 - \beta e^{4X} \cos(4\theta) - \theta x + aX )}{4 \beta e^{4X} \cos(4\theta) - a - 2tX}\quad (3)$

In [4]:
""" XisOK(X, y, n, m; t=.4, θ=θ_default(y))

    Returns true iff β*bigexp(4*X)*cos(4*θ) > max(t/2, (a+2*t*X)/4) indicating bounds for the tails of Itθ and Jtθ will be valid.
    """
function XisOK(X, y, n, m; t=.4, θ=θ_default(y))
    β = π*n^2
    a = m+y
    return β*bigexp(4*X)*cos(4*θ) > max(t/2, (a+2*t*X)/4)
end

""" Itθ_tail(X, x, y, n, m; t=.4, θ = θ_default(y))
    
    Returns a provably correct upper bound for the tail of Itθ, meaning the integral from X to ∞.
    """
function Itθ_tail(X, x, y, n, m; t=.4, θ = θ_default(y))
    XisOK(X,y,n,m,t=t,θ=θ) || error("X is too small")
    β = π*n^2
    a = m+y
   return bigexp(-t*θ^2+t*X^2-β*e^(4*X)*cos(4*θ)-θ*x+a*X)/(4*β*e^(4*X)*cos(4*θ)-a-2*t*X)
end

Itθ_tail

In [5]:
setprecision(80) do
    @show abs(Itθ(1.0,.4,1,9,lower_limit=1.0)[1])
    @show Itθ_tail(1.0, 1.0, .4, 1, 9); println()
    @show abs(Itθ(10.0,.4,1,9,lower_limit=1.0)[1])
    @show Itθ_tail(1.0, 10.0, .4, 1, 9); println()
    @show abs(Itθ(20.0,.4,1,9,lower_limit=1.0)[1])
    @show Itθ_tail(1.0, 20.0, .4, 1, 9); println()
    @show abs(Itθ(100.0,.4,1,9,lower_limit=1.0)[1])
    @show Itθ_tail(1.0, 100.0, .4, 1, 9)
return nothing
end

abs((Itθ(1.0, 0.4, 1, 9, lower_limit=1.0))[1]) = 4.4257797589751048852312986e-74
Itθ_tail(1.0, 1.0, 0.4, 1, 9) = 9.928714547542804e-74

abs((Itθ(10.0, 0.4, 1, 9, lower_limit=1.0))[1]) = 4.0237534195550521640081346e-74
Itθ_tail(1.0, 10.0, 0.4, 1, 9) = 9.022703981566085e-74

abs((Itθ(20.0, 0.4, 1, 9, lower_limit=1.0))[1]) = 3.6190168988392245809521001e-74
Itθ_tail(1.0, 20.0, 0.4, 1, 9) = 8.11265532988406e-74

abs((Itθ(100.0, 0.4, 1, 9, lower_limit=1.0))[1]) = 1.538063850976725235061819e-74
Itθ_tail(1.0, 100.0, 0.4, 1, 9) = 3.4655610420501182e-74


#### Bounding the tail of the infinite sum

Quoting from [the reference](https://terrytao.wordpress.com/2018/03/18/polymath15-sixth-thread-the-test-problem-and-beyond/), expression (3) will start decaying rapidly when X is large enough (basically one needs $\beta e^{4X}cos(4\theta)$ to become large).

For large values of n (and hence of $\beta = \pi n^2$), one can set $X=0$ and estimate the entire integral. Indeed, we conclude that if $\beta = \pi n^2$ then

$|I_{t,\theta}(b,\pi n^2)| \leq \frac{\exp( -t \theta^2 - \pi n^2 \cos(4\theta) - \theta x )}{4 \pi n^2 \cos(4\theta) - a}.$

For instance, if $\pi n_0^2 \cos(4\theta) \gt \max(\frac{t}{2}, \frac{a}{4} )$

$|\sum_{n \geq n_0} 2\pi^2 n^4 I_{t,\theta}(x-ia, \pi n^2)| \le  \frac{2\pi^2 \exp(-t \theta^2 - \theta x - \pi n_0^2 \cos(4\theta))}{4\pi n_0^2 \cos(4\theta) - a}( \frac{n_0^4}{1 - \alpha} + 
\frac{4 n_0^3 \alpha}{(1 - \alpha)^2}  + \frac{6 n_0^2 (\alpha^2+\alpha)}{(1-\alpha)^3} + \frac{4n_0 (\alpha^3+4\alpha^2+\alpha)}{(1-\alpha)^4} + \frac{\alpha^4 + 11 \alpha^3 + 11 \alpha^2 + \alpha}{(1-\alpha)^5} ) \quad (4)$

where $\alpha := e^{-\pi n_0 \cos(4\theta)}$. [Note: as rendered, $\alpha$ is easily confused with $a$ in the above expression.]

Similarly,

$|\sum_{n \geq n_0} 3\pi n^2 I_{t,\theta}(x-ia, \pi n^2)|
\leq \frac{3\pi \exp(-t \theta^2 - \theta x - \pi n_0^2 \cos(4\theta))}{4\pi n_0^2 \cos(4\theta) - a} 
( \frac{n_0^2}{1-\alpha} + \frac{2n_0 \alpha}{(1-\alpha)^2} + \frac{\alpha^2+\alpha}{(1-\alpha)^3} ),$

which allows control of the tail of the series

$H_t(z) = \frac{1}{2} \sum_{n=1}^\infty 2\pi^2 n^4 I_{t,\theta}(x-(9-y)i, \pi n^2) - 3\pi n^2 I_{t,\theta}(x-(5-y)i, \pi n^2) + 2\pi^2 n^4 \overline{I_{t,\theta}(x-(9+y)i, \pi n^2)} - 3\pi n^2 \overline{I_{t,\theta}(x-(5+y)i, \pi n^2)} \quad (0)$

In [6]:
""" minimum_n(x,y,m; t= .4, θ=θ_default(y))

    Return the minimum n for which π*n^2cos(4θ)>max(t/2,a/4), where a=m+y. Note that cos(4θ)≥0.0 for allowable θ. 
    """
function minimum_n(x,y,m; t= .4, θ=θ_default(y))
    return ceil(Int, √(max(t/2, (m+y)/4)/(π*cos(4*θ))))
end

minimum_n

In [7]:
""" series_tail_Itθ9(x,y; t=.4, θ=θ_default(y), n0=minimum_n(x,y,9, t=t, θ=θ))

    Return a provably correct upper bound on the tail |∑2*π^2*n^4*Itθ(x,y,9)|, along with n0 where summation is from n0 to ∞.
    """
function series_tail_Itθ9(x,y; t=.4, θ=θ_default(y), n0=minimum_n(x,y,9,t=t,θ=θ))
    α = bigexp(-π*n0*cos(4*θ))
    m = 9 # by definition of this series
    a = m+y
    return (2*π^2*bigexp(-t*θ^2-θ*x-π*n0^2*cos(4*θ)))/(4*π*n0^2*cos(4*θ)-a)*
            (n0^4/(1-α) + 4*n0^3*α/(1-α)^2 + 6*n0^2*(α^2 + α)/(1-α)^3 + 4*n0*(α^3+4*α^2+α)/(1-α)^4 + (α^4+11*α^3+11*α^2+α)/(1-α)^5), n0
end

series_tail_Itθ9

In [8]:
@show series_tail_Itθ9(1600, .4)
@show series_tail_Itθ9(1600, .4, n0=10)
@show series_tail_Itθ9(1, .4)
@show series_tail_Itθ9(1, .4, n0=10)
nothing

series_tail_Itθ9(1600, 0.4) = (2.074594218175116e-8, 1)
series_tail_Itθ9(1600, 0.4, n0=10) = (3.145527146658116e-142, 10)
series_tail_Itθ9(1, 0.4) = (0.5013086986691031, 1)
series_tail_Itθ9(1, 0.4, n0=10) = (7.600908682308951e-135, 10)


In [9]:
""" series_tail_Itθ5(x,y; t=.4, θ=θ_default(y), n0=minimum_n(x,y,5, t=t, θ=θ))

    Return a provably correct upper bound on the tail |∑3*π*n^2*Itθ(x,y,5)| where summation is from n0 to ∞.
    """
function series_tail_Itθ5(x,y; t=.4, θ=θ_default(y), n0=minimum_n(x,y,5,t=t,θ=θ))
    α = bigexp(-π*n0*cos(4*θ))
    m = 5 # by definition of this series
    a = m+y
    return (3*π*bigexp(-t*θ^2-θ*x-π*n0^2*cos(4*θ)))/(4*π*n0^2*cos(4*θ)-a)*
           (n0^2/(1-α) + 2*n0*α/(1-α)^2 + (α^2 + α)/(1-α)^3)
end

series_tail_Itθ5

In [10]:
""" Ht_tail(x,y; t=.4, θ=θ_default(y), n0=max(minimum_n(x,y,5,t=t,θ=θ), minimum_n(x,y,9,t=t,θ=θ))
    
    Return an upper bound for the tail of a series expressing Ht in terms of Itθ along with the index of the tail's first term.
    """
function Ht_tail(x,y; t=.4, θ=θ_default(y), n0=max(minimum_n(x,y,5,t=t,θ=θ), minimum_n(x,y,9,t=t,θ=θ)))
    return series_tail_Itθ9(x,y,t=t,θ=θ,n0=n0)[1]+series_tail_Itθ5(x,y,t=t,θ=θ,n0=n0)[1] +
               series_tail_Itθ9(x,-y,t=t,θ=θ,n0=n0)[1]+series_tail_Itθ5(x,-y,t=t,θ=θ,n0=n0)[1], n0
end

Ht_tail

In [11]:
Ht_tail(1e-6,.4,n0=10)

(1.5432475625955968e-134, 10)

## $J_{t,\theta}$

### Preliminary math

Using reference 2 and taking the derivative with respect to $z$:

$H'_t(z) = \frac{i}{2} \int_{i\theta}^{i\theta+\infty} e^{tu^2} \Phi(u) u e^{izu}\ du - \frac{i}{2} \overline{\int_{i\theta}^{i\theta+\infty} e^{tu^2} \Phi(u) u e^{i\overline{z}u}\ du},$

$ = \frac{i}{2} \sum_{n=1}^\infty 2\pi^2 n^4 J_{t,\theta}(x-(9-y)i, \pi n^2) - 3\pi n^2 J_{t,\theta}(x-(5-y)i, \pi n^2) - 2\pi^2 n^4 \overline{J_{t,\theta}(x-(9+y)i, \pi n^2)} + 3\pi n^2 \overline{J_{t,\theta}(x - (5+y)i, \pi n^2)} \quad (1)$

where $J_{t,\theta}(b,\beta) := \int_{i\theta}^{i\theta+\infty} \exp( tu^2 - \beta e^{4u} + ibu) u\ du.$ 

From $I_{t,θ}$ above we have

$\exp(tu^2-βe^{4u}+ibu) = e^{\operatorname{logmag}(σ)}(\cos(ω(σ)) + i\sin(ω(σ)))$

where $\operatorname{logmag}(σ) = - e^{4 σ} β \cos{\left (4 θ \right )} + m σ - t θ^{2} + t σ^{2} - x θ - y σ$

and $ω(σ) = \left(- e^{4 σ} β \sin{\left (4 θ \right )} + m θ + 2 t θ σ + x σ - y θ\right)$

Hence,

$u\exp(tu^2-βe^{4u}+ibu) = e^{\operatorname{logmag}(σ)} \left[ σ \cos(ω(σ))-θ \sin(ω(σ)) + i \left( σ \sin(ω(σ)) + θ \cos(ω(σ)) \right) \right]$

In [12]:
""" Jtθ(x, y, n, m; t=.4, lower_limit=big(0.0), upper_limit=big(Inf), θ = θ_default(y))

    Estimates Jtθ by adaptive Gauss-Kronrod quadrature, returning Jtθ and an error estimate. 
    NOTE: GK quadrature will fail for oscillatory arguments--larger values of x.
    
    The quadrature function uses multiprecision arithmetic if at least one limit of integration is multiprecision.

    If a very unhelpful diagnostic appears, increase precision. It is 256 bits by default. 
    """
function Jtθ(x, y, n, m; t=.4, lower_limit=big(0.0), upper_limit=big(Inf), θ = θ_default(y))
    β = π*n^2
    mag(σ) = exp(logmag(x,y,t,σ,β,θ,m))
    ω1(σ) = ω(x,y,t,σ,β,θ,m)
    return quadgk((σ)->mag(σ)*(σ*cos(ω1(σ))-θ*sin(ω1(σ))+im*(σ*sin(ω1(σ))+θ*cos(ω1(σ)))), lower_limit, upper_limit) # return quadrature, error estimate
end

Jtθ

In [13]:
setprecision(80) do
    @show Jtθ(10.0, .4, 1, 9)
    return nothing
end

Jtθ(10.0, 0.4, 1, 9) = (1.0358763987147008924125051e-04 + 2.8203784873343872948058887e-04im, 1.6932006442518676839331508e-16)


### Preliminary code for the series in $J_{t,\theta}$

Development for $J_{t,\theta}$ closely parallels that for $I_{t,\theta}$. Hence, the development below is an abbreviated parallel of the foregoing notes.

#### Bounding $J_{t,\theta}$

$J_{t,\theta,\geq X}(b,\beta) := \int_{i\theta+X}^{i\theta+\infty} \exp( tu^2 - \beta e^{4u} + ibu)u \ du.$

when, as with $I_{t,\theta}$, when X satisfies $\beta e^{4X} \cos(4\theta) \gt \max( \frac{t}{2}, \frac{a+2tX}{4})$ (equation 2 above), we have

$|J_{t,\theta,\geq X}(b,\beta)| \leq \exp( -t \theta^2 + tX^2 - \beta e^{4X} \cos(4\theta) - \theta x + aX ) (\frac{|X+i\theta|}{4 \beta e^{4X} \cos(4\theta) - a- 2tX} + \frac{1}{(4 \beta e^{4X} \cos(4\theta) - a- 2tX)^2}) \quad (6)$

In [14]:
""" Jtθ_tail(X, x, y, n, m; t=.4, θ = θ_default(y))
    
    Returns a provably correct upper bound for the tail of Jtθ, meaning the integral from X to ∞.
    """
function Jtθ_tail(X, x, y, n, m; t=.4, θ = θ_default(y))
    XisOK(X,y,n,m,t=t,θ=θ) || error("X is too small")
    β = π*n^2
    a = m+y
   return bigexp(-t*θ^2+t*X^2-β*e^(4*X)*cos(4*θ)-θ*x+a*X)*(abs(X+im*θ)/(4*β*e^(4*X)*cos(4*θ)-a-2*t*X) 
                                                           + 1/(4*β*e^(4*X)*cos(4*θ)-a-2*t*X)^2)
end

Jtθ_tail

In [15]:
@show abs(Jtθ(10.0, .4, 1, 9, lower_limit=1.0)[1])
@show Jtθ_tail(1.0, 10.0, .4, 1, 9)
nothing

abs((Jtθ(10.0, 0.4, 1, 9, lower_limit=1.0))[1]) = 4.029855724456976071528623501535664923469598728150398252405144832917306786220484e-74
Jtθ_tail(1.0, 10.0, 0.4, 1, 9) = 9.03657533814312e-74


#### Bounding the tail of the infinite sum

$|\sum_{n \geq n_0} 2\pi^2 n^4 J_{t,\theta}(x-ia, \pi n^2)| \leq 2\pi^2 \exp( -t \theta^2 - \pi n_0^2 \cos(4\theta) - \theta x )( \frac{\theta}{4 \pi n_0^2 \cos(4\theta) - a} + \frac{1}{(4 \pi n_0^2 \cos(4 \theta) - a)^2} )$

In [16]:
""" series_tail_Jtθ9(x,y; t=.4, θ=θ_default(y), n0=minimum_n(x,y,9, t=t, θ=θ))

    Returns a provably correct upper bound on the tail |∑2*π^2*n^4*Jtθ(x,y,9)| where summation is from n0 to ∞.
    """
function series_tail_Jtθ9(x,y; t=.4, θ=θ_default(y), n0=minimum_n(x,y,9,t=t,θ=θ))
    α = bigexp(-π*n0*cos(4*θ))
    m = 9 # by definition of this series
    a = m+y
    return (2*π^2*bigexp(-t*θ^2-π*n0^2*cos(4*θ)-θ*x))*(θ/(4*π*n0^2*cos(4*θ)-a) + 1/(4*π*n0^2*cos(4*θ)-a)^2)
end

series_tail_Jtθ9

In [17]:
@show series_tail_Jtθ9(1600, .4)
@show series_tail_Jtθ9(1600, .4, n0=10)
@show series_tail_Jtθ9(1, .4)
@show series_tail_Jtθ9(1, .4, n0=10)
nothing

series_tail_Jtθ9(1600, 0.4) = 3.6366847689603227e-9
series_tail_Jtθ9(1600, 0.4, n0=10) = 3.596717443750566e-148
series_tail_Jtθ9(1, 0.4) = 0.08787750843154812
series_tail_Jtθ9(1, 0.4, n0=10) = 8.691173075730924e-141


$|\sum_{n \geq n_0} 3\pi n^2 J_{t,\theta}(x-ia, \pi n^2)| \leq 3\pi \exp( -t \theta^2 - \pi n_0^2 \cos(4\theta) - \theta x )( \frac{\theta}{4 \pi n_0^2 \cos(4\theta) - a} + \frac{1}{(4 \pi n_0^2 \cos(4\theta) - a)^2} )$

In [18]:
""" series_tail_Jtθ5(x,y; t=.4, θ=θ_default(y), n0=minimum_n(x,y,5, t=t, θ=θ))

    Return a provably correct upper bound on the tail |∑3*π*n^2*Jtθ(x,y,5)| where summation is from n0 to ∞.
    """
function series_tail_Jtθ5(x,y; t=.4, θ=θ_default(y), n0=minimum_n(x,y,5,t=t,θ=θ))
    α = bigexp(-π*n0*cos(4*θ))
    m = 5 # by definition of this series
    a = m+y
    return 3*π*bigexp(-t*θ^2-π*n0^2*cos(4*θ)-θ*x)*(θ/(4*π*n0^2*cos(4*θ)-a) + 1/(4*π*n0^2*cos(4*θ)-a)^2)
end

series_tail_Jtθ5

In [19]:
""" H′t_tail(x,y; t=.4, θ=θ_default(y), n0=max(minimum_n(x,y,5,t=t,θ=θ), minimum_n(x,y,9,t=t,θ=θ))
    
    Return an upper bound for the tail of a series expressing H′t in terms of Jtθ along with the index of the tail's first term.
    """
function H′t_tail(x,y; t=.4, θ=θ_default(y), n0=max(minimum_n(x,y,5,t=t,θ=θ), minimum_n(x,y,9,t=t,θ=θ)))
    return series_tail_Jtθ9(x,y,t=t,θ=θ,n0=n0)[1]+series_tail_Jtθ5(x,y,t=t,θ=θ,n0=n0)[1] +
               series_tail_Jtθ9(x,-y,t=t,θ=θ,n0=n0)[1]+series_tail_Jtθ5(x,-y,t=t,θ=θ,n0=n0)[1], n0
end

H′t_tail

In [20]:
H′t_tail(1e-6,.4,n0=10)

(2.591869867872241e-140, 10)